## Tweet Vorverarbeitung

In diesem Notebook wird der Prozess erläutert, der die rohen Tweets vorverarbeitet. Die Tweets wurden aus der MongoDB gezogen und liegen in einem CSV-Format vor. Der Name der CSV-Datei lautet dabei immer 'streaming_tweets_' + die Anzahl der in ihr gespeicherten Tweets, im aktuellen Fall lautet der Dateiname also 'streaming_tweets_41155.csv'.  
Im ersten Schritt werden die benötigten Bibliotheken geladen und importiert, sowie eine Einstellung an der Darstellung von der Pandas geändert, die die maximale Spaltenbreite auf unendlich erhöht, damit die Tweets in einem DataFrame bei der Ausgabe vollständig angezeigt werden und nicht abgeschnitten werden, das war insbesondere bei der Entwicklung einzelner Funktionen sehr hilfreich, da man sich nicht immer die einzelnen Zellen extra ausgeben musste.  
Zudem wird hier bereits das [SentimentModel]((https://huggingface.co/oliverguhr/german-sentiment-bert). ) von oliverguhr, 'german-sentiment-bert' mittels der MySentimentModel-Klasse geladen. 

In [11]:
import pandas as pd
import numpy as np
import nltk
import re
import json
from datetime import datetime
from my_german_sentiment import MySentimentModel
import spacy
from spacy import displacy 
import matplotlib.pyplot as plt 
import numpy as np
from matplotlib.pyplot import figure
import time
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)
model = MySentimentModel(model_name = "oliverguhr/german-sentiment-bert")

### Laden der Tweets

Jetzt wird die aktuellste CSV-Datei manuell mittels der read_csv()-Funktion von Pandas eingelesen und eine Vorauswahl von interessanten Spalten definiert, von der im nächsten Schritt eine zufällige Auswahl (3 Stück) ausgegeben werden.

In [2]:
df = pd.read_csv('streaming_tweets_41155.csv', low_memory=False)
list_columns = ['_id', 'created_at', 'text', 'extended_tweet.full_text', 'user.friends_count', 'retweeted_status.source']
df[list_columns].sample(3)

,_id,created_at,text,extended_tweet.full_text,user.friends_count,retweeted_status.source
8522,1462777849769771009,Mon Nov 22 13:39:51 +0000 2021,"RT @dl_eimsbuettel: Es hat ""puff"" gemacht, und dann war sie kapott: Die grünschwarze Koalition in der #Bezirksversammlung #Eimsbüttel. Mit…",NaN,1436,"<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Web App</a>"
9862,1463127103902212098,Tue Nov 23 12:47:39 +0000 2021,Onisiwo hatte nach seiner Körperverletzung zum Glück nicht abgewunken und ist darüber hinaus Aszendent Wasserkopf.… https://t.co/bytSJp7KNG,"Onisiwo hatte nach seiner Körperverletzung zum Glück nicht abgewunken und ist darüber hinaus Aszendent Wasserkopf. Hatte Timo Horn Glück, dass er von Astrokin nicht Gelb für diesen hinterhältigen Tritt gegen den anlaufenden Mainzer bekommen hat.",1211,NaN
26697,1467537203295313938,Sun Dec 05 16:51:49 +0000 2021,"RT @sixtus: Wow! Die @DB_Bahn führt ICE-Sprinter zwischen Berlin und Köln ein, die völlig ohne Zwischenstopp durchfahren! Kein Picknick in…",NaN,842,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>"


### Filtern von Zug-Informationen

Der erste Schritt, um eine saubere Datengrundlage zu erhalten war, die Retweets zu filtern und herauszuschmeißen. Ein Retweet enthält genau denselben Inhalt wie der Originaltweet und spiegelt maximal wider, wie beliebt eine bestimmte Meinung ist. Da aber häufig prominente retweeted werden, könnte dies die Ergebnisse aber auch verfälschen, weswegen wir uns dazu entschieden haben, alle Retweets zu löschen.

Da die Tweets eine überwältigende Anzahl von Zuginformationen aufweisen, haben wir uns im nächten Schritt dazu entschieden, die größten Verursacher dieser Tweets ausfindig zu machen und Tweets, die von diesem Nutzer stammen, direkt aus dem DataFrame zu entfernen. Dazu zählen Nutzer wie metronom4me und die Deutsche Bahn und zuginfo.nrm.  
In der Ausgabe kann man sehen, dass die Gesamtanzahl der Tweets (41251) ohne Retweets nur noch 23405 beträgt und nach dem Entfernen der Zuginformationsanbietertweets nur noch 21180. Die reine Entfernung von Wiederholungen und Zuginformationen hat unseren Datensatz also bereits etwa halbiert.

In [3]:
print(f'total number of tweets: {len(df)}')
df = df[df['retweeted_status.source'].isna()]
print(f'total number of tweets without retweets: {len(df)}')
df = df[~df['user.name'].isin(['metronom4me', 
                               'metronom RE3 Hamburg Lüneburg Uelzen', 
                               'Metronom RE4 Hamburg Rotenburg Bremen',
                               'erixx.de', 
                               'eurobahn', 
                               'zuginfo.nrw',
                               'HADAG - Genau deine Elbe.',
                               'DB Regio Schleswig-Holstein'])]
print(f'total number of tweets without retweets and without train information: {len(df)}')

total number of tweets: 41251
total number of tweets without retweets: 23405
total number of tweets without retweets and without train information: 21180


### Füllen der 'full_text'-Spalte

Die Tweets werden von der Twitter-API als JSON-Format zurückgegeben. Liegt die Text-länge des Tweets über einem bestimmten Wert, wird das Feld 'Text' nur mit einem Teil des Tweets gefüllt und ein zusätzliches Feld 'Extended_tweet.full_text' angelegt, in das der gesamte Text geschrieben wird. Damit in diesem Datensatz nur eine Spalte mit Text vorhanden ist, wird eine Funktion definiert, die prüft, ob es einen Eintrag in der Spalte mit dem extended-text gibt und wenn dies der Fall ist, wird der Wert in der Spalte 'full_text' gespeichert. Gibt es keinen zusätzlichen Eintrag für 'extended_tweet.full_text' wird einfach der Text aus der 'text' Spalte übernommen.

In [4]:
def get_full_text(row):
    if isinstance(row['extended_tweet.full_text'], str):
        return row['extended_tweet.full_text']
    elif isinstance(row['text'], str):
        return row['text']
    else:
        return 'nan'

df['full_text'] = df.apply(get_full_text, axis=1)
df = df.drop(columns=['text', 'extended_tweet.full_text'])
list_columns = ['_id', 'created_at', 'full_text', 'user.friends_count']
df[list_columns].sample(5)

,_id,created_at,full_text,user.friends_count
19454,1465998829245849600,Wed Dec 01 10:58:52 +0000 2021,"@MichelTriemer @DToxikologe Hier hat man ein mini-Impfding vor 1-2 Wochen oder so aufgebaut, aber nur 200 Dosen pro Tag und nur 3 Tage in der Woche - das ""ist für uns ja noch Neuland"" - Termine waren auch instant weg (nur für 2 Wochen buchbar wg Neuland). Nun nimmt man wohl wieder das alte ZIZ in Betrieb",814
14500,1464562301592342529,Sat Nov 27 11:50:37 +0000 2021,Mein letzter LIVE-Kommentar des Jahres 😩 #LigaZwa,930
31570,1469781881759711234,Sat Dec 11 21:31:22 +0000 2021,Gut gemacht #F95 und herzlichen Glückwunsch zum Punktgewinn gegen St. Pauli. Am nächsten Freitag muss das Spiel gegen Sandhausen gewonnen werden. Bitte keine Ausreden mehr.\n\n#F95FCSP,1314
23846,1466880518406950913,Fri Dec 03 21:22:23 +0000 2021,Ist das (am) Viehmarkt?\n😂🤣😜\n\n👇👇👇,315
14541,1464570952923164676,Sat Nov 27 12:25:00 +0000 2021,"In der Zeit von 14:00-17:30 Uhr kommt es aufgrund einer #Demonstration zu #Verkehrseinschränkungen in #Mitte entlang der Strecke Am Lustgarten, Unter den Linden, Wilhelmstraße, Dorotheenstraße und Scheidemannstraße bis zum Platz der Republik.",292


### Vorbereiten des Textes auf die Sentiment Analyse

In der Beschreibung von Oliver Guhr zu seinem deutschen Sentiment Modell von BERT ist die Vorverarbeitung des Textes beschrieben, die notwendig ist, damit das Modell mit Texten arbeiten kann. Diese Normalisierung wenden wir im nächsten Schritt auf unseren Text an. 
Dabei wir unter anderem:
-   Absätze entfernt
-   Links durch den String 'URL' ersetzt
-   Sonderzeichen entfernt
-   Nummern durch Text ersetzt
-   Aufeinander folgende Leerzeichen durch einzelne ersetzt
-   Großschreibung durch Kleinschreibung ersetzt

Der Unterschied zwischen dem gereinigten und dem originalen Text wird durch die Ausgabe nebeneinander dargestellt.

In [5]:
df['clean_text'] = df['full_text'].apply(model.clean_text)
df[['clean_text', 'full_text']].sample(3)

,clean_text,full_text
2129,shirindavid bramfeld storys gibt richtig gänsehautvorallem die lines über die yt zeit,@ShirinDavid Bramfeld Storys gibt richtig Gänsehaut…vorallem die lines über die YT Zeit 🔥
22811,unfall in meckenheim aktuell was ist heute passiert die polizeidirektion neustadtweinstraße informiert über polizeimeldungen von heute url hält sie auf dem laufenden zu unfall brand und verbrechensmeldungen in ihrer region url,"Unfall in Meckenheim aktuell: Was ist heute passiert? Die Polizeidirektion Neustadt/Weinstraße informiert über Polizeimeldungen von heute. https://t.co/qCm74oFEdW hält Sie auf dem Laufenden zu Unfall-, Brand- und Verbrechensmeldungen in Ihrer Region. https://t.co/CGjYZEW36r"
5081,momogibsnet zwei grlieder meine tochter gehört zu denen die buchstäblich vom system gesprengt wurden und die schwere und vielzahl der ihr auch in der jugendhilfe erst vinzenzwerk handorf dann heiki hamm zugefügten seelischen verletzungen nicht verkraftet hat tod mit zwei fünf jahren,"@momogibsnet2 @g_r_lieder Meine Tochter gehört zu denen, die buchstäblich vom System ""gesprengt"" wurden und die Schwere und Vielzahl der ihr auch in der Jugendhilfe (erst Vinzenzwerk Handorf / dann Heiki Hamm) zugefügten seelischen Verletzungen nicht verkraftet hat. Tod mit 25 Jahren."


### Zuordnung der Tweets zu einem Bezirk

Die Tweets wurden anhand von Filterwörtern gespeichert, die entweder einem Bezirk oder einem Park entsprechen. Die Twitter API gibt bei den zurückgegebenen Tweets aber nicht an, welches Filterwort für die Auswahl des Tweets verantwortlich war, wodurch eine Zuordnung der Bezirke zu den Tweets notwendig ist.  
Dazu haben wir die Datei, die Stadtteile und Parks einem Bezirk zuordnen als JSON-Datei geladen und für jeden Tweet geprüft, zu welchem Bezirk dieser gehört und in der Spalte 'district' gespeichert. Falls mehrere Filterwörter dafür gesorgt haben, dass ein Tweet ausgewählt wurde, wird hier immer das erste 'Match' genutzt.  
Falls der Fall auftritt, dass kein Bezirk gefunden wurde, wird dieser Tweets gelöscht. Hierfür gibt es auch eine Erklärung, Twitter durchsucht bei der Streaming-Funktion der API auch die in dem Tweet erwähnten URLs und stößt dabei in der Überschrift ab und zu auf passende Filterwörter, die wir in diesem Fall aber nicht zusätzlich beachtet haben.

In [6]:
list_columns = ['_id', 'created_at', 'full_text', 'user.friends_count', 'district', 'user.name', 'user.description']

with open('match_dict.json', 'r') as fp:
    match_dict = json.load(fp)

def matching_district(doc):
    for key in match_dict.keys():
        if re.search(key, doc, re.IGNORECASE):
            return match_dict[key]


df['district'] = df['full_text'].apply(matching_district)
df = df[df['district'].notna()]
df[list_columns].sample(3)

,_id,created_at,full_text,user.friends_count,district,user.name,user.description
28580,1468145380370132992,Tue Dec 07 09:08:30 +0000 2021,New #vacancy #FrenchHorn @StaatstheaterN\n\nStaatsphilharmonie Nürnberg\n\n1. Solo-Horn 100% (1) – Permanent\n\nhttps://t.co/oGm9IDWvNg https://t.co/VIb4XbGZ9D,145,Horn,muvac,muvac is an online service that facilitates the application procedure in classical music institutions for everyone involved in the process.
16152,1465019241653059591,Sun Nov 28 18:06:20 +0000 2021,@swag1321 Yes in bergedorf,1425,Bergedorf,Tarik Abi,"Du völlig Wahnsinniger! Du unrealistischer, Bildungsresistenter, Intelligenzallergiker. Alhamdulilah Muslim 🕋📿"
3366,1461735028896014338,Fri Nov 19 16:36:03 +0000 2021,"Freitag 17:35 und noch kein @GeniusDeu ""Was ist euer Lieblingsrelease?"" tweet deshalb fange ich an mit Shirin David - Bramfeld Storys",69,Bramfeld,🅱️reter,"18 | Twitch Konsument | Fußball Fan | Musik Connaisseur | Memer\nonly the voices in my head are (they/them)\nManchmal glaub ich, dass ich lustig bin"


### Zusätzliches Filtern von mehrdeutigen Bezirken

Manche Bezirke sind nicht eindeutig Hamburg zuzuordnen, so wird teilweise über Altstadt geredet, aber die Jerusalemer Altstadt gemeint. Manche Tweets beziehen sich beim Thema Horn nicht auf Hamburg Horn, sondern den Fußballer Jannes oder Timo Horn vom 1. FC Köln. Analog wird teilweise auch beim Bezirk Hamm der deutschen Sprache Unausprechliches angetan, wenn der Inhalt des Tweets 'Wir hamm ein Problem' lautet.  
Um bei diesen Tweets für Sicherheit zu sorgen, wird in Tweets , dessen Bezirk einem der ausgewählten entspricht, und den Nutzernamen und Beschreibungen zusätzlich nach dem Stichwort 'Hamburg' oder 'hh' gesucht, wird keines dieser beiden Ausdrücke gefunden, wird der Tweet aus dem DataFrame entfernt.

In [7]:

df['user.name'] = df['user.name'].astype(str)
df['user.description'] = df['user.description'].astype(str)
all_keywords = list(set(match_dict.keys()))
unique_keywords = [key for key in all_keywords if key not in ['Altstadt','Horn', 'Neustadt', 'Neuland', 'Hamm', 'Marienthal']]
unique_keywords.extend(['hamburg', 'hh'])
unique_keywords = [key.lower() for key in unique_keywords]

df['is_hamburg'] = [True if (any(x in text for x in unique_keywords) | ('hamburg' in (name+desc).lower())) else False for text, name, desc in zip( df['clean_text'], df['user.name'], df['user.description'])]

df['is_hamburg'].value_counts()
df_ham = df[df.is_hamburg]

### Ist das Filterwort ein Ort?

Als letzter Filterschritt wird nun mithilfe von [Spacy](https://spacy.io/), einem neuronalen Sprachmodell jeder Tweet analysiert und die Entitäten ausgegeben. Entspricht der als Filterwort deklarierte Bezirk jetzt zusätzlich einem von Spacy erkannten Ort, gilt der Tweet als sicher Hamburg zuzuordnen und wird im DataFrame behalten.

In [8]:
nlp = spacy.load('de_core_news_sm')
# Text with nlp
is_loc = []
for text, district in zip(df_ham['clean_text'], df_ham['district']):
    doc = nlp(text)    
    # Display Entities
    list_loc = [(token.text) for token in doc if token.ent_type_ == 'LOC']
    loc_string = ' '.join([str(item) for item in list_loc])
    if re.search(district, loc_string, re.IGNORECASE):
        is_loc.append(True)
        #displacy.render(doc, style="ent")
    else:
        is_loc.append(False)
df_ham['district_is_loc'] = is_loc
list_columns.append('district_is_loc')
df_ham['district_is_loc'].value_counts()
df_loc = df_ham[df_ham['district_is_loc']]
df_loc[list_columns].sample(3)

,_id,created_at,full_text,user.friends_count,district,user.name,user.description,district_is_loc
18606,1465713640946819073,Tue Nov 30 16:05:38 +0000 2021,"Die stadtplanerische ""Leistung"", die dieser Monstrosität, voller Hohn Elbtower getauft, zugrundeliegt, testet meinen Glauben an Gewaltfreiheit.\nDie HafenCity ist bereits unansehnlich, aber muss man dem gesamten Stadtbild mit diesem überdimensionierten Dödel das Gleiche antun? https://t.co/bhz5PJs1TV",319,HafenCity,Tim Nanns,Macht was mit Politik.,True
13084,1464183928428847106,Fri Nov 26 10:47:06 +0000 2021,„Härtefallkommission der Hamburgischen Bürgerschaft: Bleiberecht für fünf Kinder aus Hamburg-Wilhelmsburg und ihre Mutter!” - Jetzt unterschreiben! https://t.co/AgpdR7xbgu via @ChangeGER,137,Wilhelmsburg,Klaus D.aus B.,Antifaschist! \nRenitenter Hauptstadtrentner ✊\nAPO Opa.,True
30253,1468591749325012997,Wed Dec 08 14:42:12 +0000 2021,@PriiMeCoD @CChilloo @Crizofy Meram in Billstedt ist eine 10/10,202,Billstedt,‏ً,@spayzhJ @stojzy @ysleon11 @maximyrn @t6mmy_,True


### Sentiment Analyse

Der letzte Schritt in diesem Notebook und gleichermaßen der interessanteste Schritt ist die Analyse des Sentiments in einem Tweets. Dafür wird das oben bereits erwähnt deutsche Sentiment Analysemodell von Oliver Guhr verwendet, das ein fein getuntes Modell von [BERT](https://arxiv.org/abs/1810.04805) ist.  
BERT steht für Bidirectional Encoder Representations from Transformers und stellt ein Modell dar, das anhand ungelabelter Daten durch Kontextbasiertes Training für unterschiedliche Aufgaben vortrainiert wurde. Um BERT für zielgerichtete Anwendungen anhand von gelabelten Daten zu trainieren, reicht eine Ausgabeschicht für das neuronale Netz, die mittels der Trainingsdaten feingetuned wird.  
Wir nutzen im Folgenden das bereit für die deutsche Sprache trainierte Modell, das auch anhand von Twitter-Daten trainiert wurde, um Sentimentalitäten in Tweets zu erkennen und mit einem von drei Labels zu versehen: 'Positiv', 'Neutral', 'Negativ'.  
Da das Analysieren der Tweets bereits sehr leistungshungrig ist, wird die Analyse batchweise durchgeführt, also immer 100 Tweets auf einmal in der BERT-Modell gegeben, um den Prozessor nicht zu überlasten.


In [24]:
predictions = []
step_size = 100

exec_time_avg = 10
for i in np.arange(0, len(df_loc), step_size):
    start = time.time()
    for prediction in model.predict_sentiment(list(df_loc.iloc[i:i+step_size]['clean_text'])):
        predictions.append(prediction)
    exec_time_one = round(time.time()-start,2)
    print(f'{len(predictions)}/{len(df_loc)} Tweets analysed. Execution Time: {exec_time_one}s.')
df_loc['sentiment'] = [sentiment[0] for sentiment in predictions]
df_loc['sentiment_score'] = [sentiment[1] for sentiment in predictions]
df_loc.to_csv('df_sent.csv')
df_loc[list_columns].sample(3)

100/2594 Tweets analysed. Execution Time: 8.95s.
200/2594 Tweets analysed. Execution Time: 13.94s.
300/2594 Tweets analysed. Execution Time: 11.3s.
400/2594 Tweets analysed. Execution Time: 13.9s.
500/2594 Tweets analysed. Execution Time: 9.2s.
600/2594 Tweets analysed. Execution Time: 12.44s.
700/2594 Tweets analysed. Execution Time: 8.71s.
800/2594 Tweets analysed. Execution Time: 9.54s.
900/2594 Tweets analysed. Execution Time: 11.59s.
1000/2594 Tweets analysed. Execution Time: 9.04s.
1100/2594 Tweets analysed. Execution Time: 9.54s.
1200/2594 Tweets analysed. Execution Time: 10.35s.
1300/2594 Tweets analysed. Execution Time: 8.92s.
1400/2594 Tweets analysed. Execution Time: 9.58s.
1500/2594 Tweets analysed. Execution Time: 10.56s.
1600/2594 Tweets analysed. Execution Time: 7.93s.
1700/2594 Tweets analysed. Execution Time: 12.27s.
1800/2594 Tweets analysed. Execution Time: 42.96s.
1900/2594 Tweets analysed. Execution Time: 11.4s.
2000/2594 Tweets analysed. Execution Time: 12.31s.
21

,_id,created_at,full_text,user.friends_count,district,user.name,user.description,district_is_loc
12448,1463962323647881221,Thu Nov 25 20:06:31 +0000 2021,@FidelZastro ich stell mich in eppendorf vor die klinik wer nicht lächelt bekommt nur ein fernsehprogramm und zwar vollständig von @KaiHermy produziert und moderiert,481,Eppendorf,Rolf Coptaire-Bärenklau,redlicher kl1user für imbissstubencontent mofafrisierkniffe tabaksgenuss und wagenhebersozialdemokratie hier nur geschäftlich rt na clear endorsement ihr jocks,True
31798,1469923723608305664,Sun Dec 12 06:55:00 +0000 2021,#24malWandsbek #Adventskalender2021\nZum dritten Advent öffnen wir das Türchen Nummer 12 mit dem Stadtteil #Hummelsbüttel!\nHat der Stadtteil wirklich etwas mit Hummeln zu tun? 🐝\nUnd wo könnte man diesen Sonntag einen Ausflug machen?⬇️\n#unserWandsbek https://t.co/zQmzL21NGQ,470,Hummelsbüttel,Bezirksamt Wandsbek,"Hier berichtet das Bezirksamt über alles, was es in Hamburgs Nordosten bewegt. Unmarkierte Bilder von uns. http://www.hamburg.de/impressum/370090/impressum-fhh/",True
30557,1468648620643823616,Wed Dec 08 18:28:11 +0000 2021,"@stefan_uhlmann @jayelkay13 @MickyBeisenherz Ist er nicht. Scholz wurde in der Christianskirche in Hamburg-Ottensen getauft, trat später aber aus der evangelischen Kirche aus und ist seither konfessionslos.",559,Ottensen,Andreas Klett,Account Executive * Addicted to technology * Passionate Marketeer & Salesman * (Re-)Tweets express my personal opinion,True
